# About this notebook

This notebook explores fire severity as classification problem using integrated dataset with 40 PCA and provides results based on two threshold recomneded to seperate labels 

## Loading the fire predictions

In [243]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

In [244]:
%%time
import pandas as pd
import pyarrow.parquet as pq
import s3fs
import boto3
import io

Wall time: 0 ns


In [245]:
# Accessing S3 bucket with fireonly integrateddata wiht pca40
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
fireonly_url = 'wildfire_capstone/integratedData/integratedData.pca40_and_firesonly.parquet.gz/'
s3_keys = [item.key
           for item in s3_resource.Bucket('dse-cohort5-group5').objects.filter(Prefix=fireonly_url)
           if item.key.endswith('.parquet')]
display(s3_keys[:3])

['wildfire_capstone/integratedData/integratedData.pca40_and_firesonly.parquet.gz/part-00000-f1d8e022-8b57-43dc-a0dd-592bc9adf2ad-c000.gz.parquet',
 'wildfire_capstone/integratedData/integratedData.pca40_and_firesonly.parquet.gz/part-00001-f1d8e022-8b57-43dc-a0dd-592bc9adf2ad-c000.gz.parquet',
 'wildfire_capstone/integratedData/integratedData.pca40_and_firesonly.parquet.gz/part-00002-f1d8e022-8b57-43dc-a0dd-592bc9adf2ad-c000.gz.parquet']

In [246]:
len(s3_keys)

200

In [248]:
##anon=False
s3 = s3fs.S3FileSystem()   
s3_url = 'dse-cohort5-group5/{}'
dfs = [pq.ParquetDataset(s3_url.format(key), filesystem=s3).read_pandas().to_pandas() for key in s3_keys]


In [249]:
num_rows = sum(len(df) for df in dfs)
num_rows

2167

In [253]:
# Concat all dataframes into combined dataframe
dff = pd.concat(dfs)
dff.head()

,date,latitude,longitude,fire_occurred,acres_burned,year,month,day,pcaFeaturesArr[0],pcaFeaturesArr[1],...,pcaFeaturesArr[30],pcaFeaturesArr[31],pcaFeaturesArr[32],pcaFeaturesArr[33],pcaFeaturesArr[34],pcaFeaturesArr[35],pcaFeaturesArr[36],pcaFeaturesArr[37],pcaFeaturesArr[38],pcaFeaturesArr[39]
0,1067040000000000000,32.775000,-116.933333,1,270686.000000,2003,10,25,-1.279795,12.545055,...,-0.546615,0.039315,-1.049986,0.148678,0.140634,-0.628120,-0.615347,0.606489,-2.682956,-0.000420
1,1067040000000000000,32.941667,-116.766667,1,270686.000000,2003,10,25,8.553702,6.594267,...,-1.024007,0.032058,-0.831248,0.387239,0.570735,0.139129,0.109658,-0.161319,-1.332964,0.024806
2,1192924800000000000,32.691667,-116.891667,1,90728.328125,2007,10,21,1.144086,2.739561,...,-0.574740,0.223633,-1.104973,-0.054726,0.818893,0.589548,-0.112022,0.331131,0.078054,-0.036084
3,1193011200000000000,33.191667,-117.016667,1,164038.218000,2007,10,22,1.415704,1.031003,...,-0.125726,-1.066130,-0.631703,1.564491,0.816761,-0.657401,0.711758,-0.141134,0.443211,-0.159693
4,1193097600000000000,33.066667,-116.850000,1,108911.685000,2007,10,23,3.869830,-3.855695,...,-0.571720,-0.789562,-0.618795,0.994336,1.086224,-1.182717,-0.218400,0.231799,-0.308896,0.188512


In [254]:
dff.head()

,date,latitude,longitude,fire_occurred,acres_burned,year,month,day,pcaFeaturesArr[0],pcaFeaturesArr[1],...,pcaFeaturesArr[30],pcaFeaturesArr[31],pcaFeaturesArr[32],pcaFeaturesArr[33],pcaFeaturesArr[34],pcaFeaturesArr[35],pcaFeaturesArr[36],pcaFeaturesArr[37],pcaFeaturesArr[38],pcaFeaturesArr[39]
0,1067040000000000000,32.775000,-116.933333,1,270686.000000,2003,10,25,-1.279795,12.545055,...,-0.546615,0.039315,-1.049986,0.148678,0.140634,-0.628120,-0.615347,0.606489,-2.682956,-0.000420
1,1067040000000000000,32.941667,-116.766667,1,270686.000000,2003,10,25,8.553702,6.594267,...,-1.024007,0.032058,-0.831248,0.387239,0.570735,0.139129,0.109658,-0.161319,-1.332964,0.024806
2,1192924800000000000,32.691667,-116.891667,1,90728.328125,2007,10,21,1.144086,2.739561,...,-0.574740,0.223633,-1.104973,-0.054726,0.818893,0.589548,-0.112022,0.331131,0.078054,-0.036084
3,1193011200000000000,33.191667,-117.016667,1,164038.218000,2007,10,22,1.415704,1.031003,...,-0.125726,-1.066130,-0.631703,1.564491,0.816761,-0.657401,0.711758,-0.141134,0.443211,-0.159693
4,1193097600000000000,33.066667,-116.850000,1,108911.685000,2007,10,23,3.869830,-3.855695,...,-0.571720,-0.789562,-0.618795,0.994336,1.086224,-1.182717,-0.218400,0.231799,-0.308896,0.188512


In [255]:
# No need for fire occurred as this data is for fire only 
dff=dff.drop(['date', 'fire_occurred'], axis=1)
dff.shape

(2167, 46)

In [257]:
# We will use threashold as suggested by SDGE
# First assumption, these threshold of small, medium and large fires are derived based on congresssional research study
dfm=dff
p1= 0
p2=20
p3=250
c0=dfm['acres_burned']>p1
c1=dfm['acres_burned'] <=p2
c2=dfm['acres_burned'] >p2
c3 = dfm['acres_burned'] <=p3
c4 =dfm['acres_burned'] > p3
dfm1= dfm[c0 & c1]
dfm2=dfm[c2 & c3]
dfm3=dfm[c4]
# Lable the output as 0 for small, 1 for meidum and 2 for large fires 
dfm1['label'] = 0
dfm2['label'] = 1
dfm3['label'] =2
print('Number of fires belongs to small fire (class 0) = ',dfm1.count()[1])
print('Number of fires belongs to Medium fire (class 1) = ',dfm2.count()[1])
print('Number of fires belongs to large fire (class 2) = ',dfm3.count()[1])
dfmt=pd.concat([dfm1, dfm2, dfm3])
dfmt.head()

Number of fires belongs to small fire (class 0) =  91
Number of fires belongs to Medium fire (class 1) =  222
Number of fires belongs to large fire (class 2) =  1854


,latitude,longitude,acres_burned,year,month,day,pcaFeaturesArr[0],pcaFeaturesArr[1],pcaFeaturesArr[2],pcaFeaturesArr[3],...,pcaFeaturesArr[31],pcaFeaturesArr[32],pcaFeaturesArr[33],pcaFeaturesArr[34],pcaFeaturesArr[35],pcaFeaturesArr[36],pcaFeaturesArr[37],pcaFeaturesArr[38],pcaFeaturesArr[39],label
1,33.275000,-116.975000,8.239795,2005,07,15,6.170872,11.393714,7.801698,-0.712762,...,1.142377,-0.643994,0.448114,0.125363,-0.579346,-0.767267,-0.094342,1.519810,0.036952,0
11,33.400000,-116.683333,1.480000,2012,08,19,15.368121,-0.376633,0.505262,-8.997285,...,0.450572,0.015242,1.683029,-0.165310,1.039794,-1.307665,1.758082,-1.817098,0.839298,0
3,32.525000,-116.891667,11.372368,2012,05,17,-6.545495,7.387253,9.617247,-0.088458,...,-0.638185,-0.662343,0.155991,0.835918,0.198035,-1.422496,1.475828,-0.369046,1.060529,0
4,32.608333,-116.850000,19.389981,2012,07,13,8.412544,9.050481,7.806521,1.989935,...,-0.364202,0.362691,0.422787,-0.654015,-0.167456,0.653295,-0.615254,-0.165257,-0.529173,0
6,32.650000,-116.558333,17.105000,2007,07,09,17.933396,1.766535,11.472100,4.674402,...,-0.550919,-1.054951,1.361475,0.361002,0.371563,0.155571,-1.153063,0.301566,-0.280857,0


In [258]:
# Want to check which year has similar samples distribution 
yy=dfmt['year'].unique()
for y in yy:
    t=dfmt[dfmt['year']==y]
    v=t['label'].values
    uniqueValues, occurCount = np.unique(v, return_counts=True)
    
    print(y,uniqueValues, occurCount)

2005 [0 1 2] [15 32 10]
2012 [0 1 2] [ 23  12 113]
2007 [0 1 2] [  11   12 1054]
2004 [0 1 2] [13 15 25]
2014 [0 1 2] [  4   9 121]
2013 [0 1 2] [ 5 11 72]
2009 [0 1 2] [ 3 10  2]
2006 [0 1 2] [10 10 53]
2010 [0 1 2] [ 3 14 14]
2015 [0 1 2] [ 1 10  4]
2008 [0 1] [ 1 27]
2003 [0 1 2] [  1   7 166]
2018 [0 1 2] [1 6 5]
2002 [1 2] [17 58]
2011 [1 2] [19 64]
2017 [1 2] [ 7 33]
2016 [1 2] [ 2 45]
2001 [1 2] [ 2 15]


In [259]:
# Get features with all 40 PCA plus latitude, longitude, year, month and year
dfmr=dfmt.drop(['acres_burned','label'], axis=1)
dfmt=dfmt.drop(['acres_burned'], axis=1)
#dfmt=dfmt.drop(['date'], axis=1)
features=dfmr.columns
features

Index(['latitude', 'longitude', 'year', 'month', 'day', 'pcaFeaturesArr[0]',
       'pcaFeaturesArr[1]', 'pcaFeaturesArr[2]', 'pcaFeaturesArr[3]',
       'pcaFeaturesArr[4]', 'pcaFeaturesArr[5]', 'pcaFeaturesArr[6]',
       'pcaFeaturesArr[7]', 'pcaFeaturesArr[8]', 'pcaFeaturesArr[9]',
       'pcaFeaturesArr[10]', 'pcaFeaturesArr[11]', 'pcaFeaturesArr[12]',
       'pcaFeaturesArr[13]', 'pcaFeaturesArr[14]', 'pcaFeaturesArr[15]',
       'pcaFeaturesArr[16]', 'pcaFeaturesArr[17]', 'pcaFeaturesArr[18]',
       'pcaFeaturesArr[19]', 'pcaFeaturesArr[20]', 'pcaFeaturesArr[21]',
       'pcaFeaturesArr[22]', 'pcaFeaturesArr[23]', 'pcaFeaturesArr[24]',
       'pcaFeaturesArr[25]', 'pcaFeaturesArr[26]', 'pcaFeaturesArr[27]',
       'pcaFeaturesArr[28]', 'pcaFeaturesArr[29]', 'pcaFeaturesArr[30]',
       'pcaFeaturesArr[31]', 'pcaFeaturesArr[32]', 'pcaFeaturesArr[33]',
       'pcaFeaturesArr[34]', 'pcaFeaturesArr[35]', 'pcaFeaturesArr[36]',
       'pcaFeaturesArr[37]', 'pcaFeaturesArr[38]', 'pcaF

In [267]:
#Remove 2004 as used a validation set for rest of years 
yyu=np.delete(yy,3)
yyu=np.sort(yyu)
yyu

array(['2001', '2002', '2003', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018'], dtype=object)

In [268]:
# Calculate the accuracy of classes for each year for last 20 years. some years dont have any fires
# Create neasted dictionary to cover each year with accuracy , confusion matrix and classificaiton report
# Results include output which has actual and predicated classes plus all features for test dataset 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import PredefinedSplit
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
output=[]
accuracy=[]
conf_matrx=[]
class_report=[]
grid_parametrs=[]
d={}

for i,year in enumerate(yyu):    
    dfmtt1=dfmt[dfmt['year']==year]
    print('Number of total fires of {} = '.format(year), dfmtt1.shape[0])
    dfmttest=pd.concat([dfmtt1])
    #Shuffle dataset with random state so it can be reproduced 
    dfmttest = dfmttest.sample(frac=1, axis=0, random_state=10).reset_index(drop=True)
    xt=dfmttest[features]
    yt=dfmttest['label']
    uniqueValues, occurCount = np.unique(yt, return_counts=True)
    print('The number of unique values of labels and their occurrences for test data of {} = '.format(year),uniqueValues, occurCount)
    dfmtt=dfmt[(dfmt['year'] != year)]
    dfmtt = dfmtt.sample(frac=1, axis=0, random_state=10).reset_index(drop=True)
    print('Number of total fires of rest of years = ', dfmtt.shape[0])
    x=dfmtt[features]
    y=dfmtt['label']
    uniqueValues, occurCount = np.unique(y, return_counts=True)
    print('The number of unique values of labels and their occurrences for traning data of rest of years= ',uniqueValues, occurCount)
    # use 2004 as validation dataset to tune the parameters 
    dfmttain=dfmt[dfmt['year'] == '2004']
    ytrain=dfmttain['label'].values
    # Focus on randomforesetclassifier as it always outperform  Logistic regression, Xgboost and SVM 
    clf=RandomForestClassifier()
    ps = PredefinedSplit(ytrain)
    parameters = {'n_estimators': [15,20,25,50,100], 'max_depth' : [5,10,15,20,25,30],'min_samples_split':[1,2,3,4,6], 
                  'min_samples_leaf':[1,2,3,5,7],'random_state' : [5]}
    CV_rfc = GridSearchCV(clf , parameters,scoring= 'f1_micro', cv=ps)
    CV_rfc.fit(x, y)
    y_pred=CV_rfc.predict(xt)
    ypd=pd.DataFrame(y_pred, columns=['PClass'])
    ytd=pd.Series.to_frame(yt, name='AClass')
    outputdf=pd.concat([xt,ypd,ytd],axis=1)
    output.append(outputdf)
    vaild_acc= metrics.f1_score(yt, y_pred, average='micro')
    print('The validation accuracy of year {} is= {} '.format(year,vaild_acc))
    accuracy.append(vaild_acc)
    conf_mat = confusion_matrix(yt, y_pred)
    vaild_acc1 = classification_report(yt, y_pred)
    conf_matrx.append(conf_mat)
    class_report.append(vaild_acc1)
    grid_parametrs.append(CV_rfc.best_params_)
    d[year]={}
    d[year]['output']=output[i]
    d[year]['accuracy']=accuracy[i]
    d[year]['conf_matrx']=conf_matrx[i]
    d[year]['classification_report']=class_report[i]
    d[year]['grid_parametrs']=grid_parametrs[i]

    

Number of total fires of 2001 =  17
The number of unique values of labels and their occurrences for test data of 2001 =  [1 2] [ 2 15]
Number of total fires of rest of years =  2150
The number of unique values of labels and their occurrences for traning data of rest of years=  [0 1 2] [  91  220 1839]
The validation accuracy of year 2001 is= 
Number of total fires of 2002 =  75
The number of unique values of labels and their occurrences for test data of 2002 =  [1 2] [17 58]
Number of total fires of rest of years =  2092
The number of unique values of labels and their occurrences for traning data of rest of years=  [0 1 2] [  91  205 1796]
The validation accuracy of year 2002 is= 
Number of total fires of 2003 =  174
The number of unique values of labels and their occurrences for test data of 2003 =  [0 1 2] [  1   7 166]
Number of total fires of rest of years =  1993
The number of unique values of labels and their occurrences for traning data of rest of years=  [0 1 2] [  90  215 1688

In [270]:
d

{'2001': {'output':      latitude   longitude  year month day  pcaFeaturesArr[0]  \
  0   32.775000 -116.808333  2001    01  03          -1.537802   
  1   32.816667 -116.683333  2001    01  03           4.118823   
  2   32.733333 -116.766667  2001    01  03           0.858958   
  3   32.775000 -116.766667  2001    01  03           0.602554   
  4   32.775000 -116.850000  2001    01  03          -2.640532   
  5   32.775000 -116.725000  2001    01  03           1.411207   
  6   32.566667 -116.641667  2001    12  08          -7.496800   
  7   32.775000 -116.683333  2001    01  03           3.693478   
  8   32.733333 -116.808333  2001    01  03          -1.608726   
  9   32.608333 -116.600000  2001    12  08          -7.557506   
  10  32.816667 -116.725000  2001    01  03           2.578765   
  11  32.816667 -116.766667  2001    01  03           0.863229   
  12  32.733333 -116.975000  2001    07  20          -5.479472   
  13  33.566667 -116.933333  2001    07  09          12.74

In [272]:
for y in yyu:
    print('The accuracy of year {} is {}'.format(y,d[y]['accuracy']))


The accuracy of year 2001 is 0.8235294117647058
The accuracy of year 2002 is 0.7866666666666666
The accuracy of year 2003 is 0.9655172413793104
The accuracy of year 2005 is 0.24561403508771928
The accuracy of year 2006 is 0.7123287671232877
The accuracy of year 2007 is 0.9795728876508821
The accuracy of year 2008 is 0.42857142857142855
The accuracy of year 2009 is 0.5333333333333333
The accuracy of year 2010 is 0.45161290322580644
The accuracy of year 2011 is 0.7951807228915663
The accuracy of year 2012 is 0.7567567567567567
The accuracy of year 2013 is 0.8522727272727273
The accuracy of year 2014 is 0.9104477611940298
The accuracy of year 2015 is 0.3333333333333333
The accuracy of year 2016 is 0.9787234042553191
The accuracy of year 2017 is 0.85
The accuracy of year 2018 is 0.4166666666666667


In [286]:
dfst=[d[y]['output'] for y in yyu ]
dfst=pd.concat(dfst)
dfst.shape

(2114, 47)

In [287]:
dfst.to_parquet('fire_severity.parquet')

In [288]:
dfst.write.parquet(
    's3a://dse-cohort5-group5/wildfire_capstone/fire_severity_classification_prediction/fire_severity_classification_result.parquet.gz',
    partitionBy=["year"],
    mode="overwrite",
    compression='gzip')

AttributeError: 'DataFrame' object has no attribute 'write'

In [290]:
dfst.to_csv('output.csv')